# Questo notebook costruisce i vari csv ripulendo i dati in modo che siano pronti per essere processati

In [1]:
import json
import csv
from datetime import datetime
import numpy as np
import pandas as pd
from os import listdir

types = ['profile.json', 'isee.json', 'exams.booked.json', 'exams.taken.json', 'diplomas.json']  # Lista dei file da aprire per ogni studente

vProfile = ["annoAccaCors", "annoCorso", "annoDiNascita", "cittadinanza", "codCorso", "creditiTotali", "erasmus", "facolta", "flagIdentificato", "isMedicina",
            "luogoDiNascita", "nazioneNascita", "primaIscr", "sesso", "tipoCorso", "tipoIscrizione", "tipoStudente", "ultIscr"]
vIsee = ["nonDichiaro", "valoreIntero"]
vBooked = ["cfuPrenotati", "ultPren"]
vTaken = ["cfuTake", "ultSup", "mediaVoto"]  # Esito -->"valoreNonNominale"
vDiplomas = ["voto", "diplomando", "codiTiso"]
DELTA = 120  # in numero di giorni, 182 = 6 mesi

######### DA CAMBIARE CON PERCORSO VERO #########################
# Vero: '/home/andrea/Scrivania/Andrea/Scuola/Triennale/Tirocinio/FullDS'
p = '../FullDS/'
N_STUDENTS = max([int(el.split('.')[0]) for el in listdir(p)]) + 1

In [2]:
N_STUDENTS

102529

### Funzioni necessarie a raggruppare i dati sparsi

In [7]:
def diff_month(d, g=1):
    # granularity g = 0 -> days
    #               = 1 -> months
    #               = 2  -> years 
    if g == 0:
        return abs((datetime.today() - d)).days
    elif g == 1:    
        return round(abs((datetime.today() - d)).days / 30) # Return the difference in months
    return round(abs((datetime.today() - d)).days / 365) # Return the difference in years

In [4]:
def take_info_booked(data):
    studentInfo = []
    if data['appelli'] != []:
        totCFU = 0
        lastDate = []
        for el in data['appelli']:
            totCFU += el['crediti']
            lastDate.append(datetime.strptime(
                el['dataAppe'], '%d/%m/%Y')) if el['dataAppe'] != None else None
        studentInfo.append(totCFU)
        lastDate.sort()
        studentInfo.append(diff_month(lastDate[-1])) if lastDate != [
        ] else None
    else:
        return [0, 6] # In 6 month booked exam will be recorded
    return studentInfo

def take_info_profile(data, v):
    v_countries = ["AUSTRIA", "BELGIO", "REPUBBLICA di BULGARIA", "REPUBBLICA di CIPRO", "CROAZIA", "DANIMARCA", "ESTONIA", "FINLANDIA", "FRANCIA", "GERMANIA",
            "REPUBBLICA FEDERALE TEDESCA", "GRECIA", "IRLANDA", "LETTONIA", "LITUANIA", "LUSSEMBURGO", "MALTA", "PAES BASSI", "POLONIA", "PORTOGALLO", "REPUBBLICA CECA",
            "ROMANIA", "SLOVACCHIA", "SLOVENIA", "SPAGNA", "SVEZIA", "UNGHERIA"]
    studentInfo = []
    if data != []:
        for el in v:
            # Cittadinanza a grana media, togliere questo if per tornare all'originale
            if el == "cittadinanza":
                if data['nazioneNascita'] == "ITALIA":
                    studentInfo.append("ITA")
                elif data['nazioneNascita'] in v_countries:
                    studentInfo.append("UE")
                else:
                    studentInfo.append("Extra-UE")
            elif el == "primaIscr" or el == "ultIscr":
                # Trasformata in: quanti anni fa è stata fatta l'iscrizione?
                studentInfo.append((2022 - int(data[el].split('/')[0]))) if data[el] != '' else studentInfo.append('None')
            elif el == "annoDiNascita":
                studentInfo.append(2022 - int(data[el]))
            else:
                studentInfo.append(data[el])
    else:
        for el in v:
            studentInfo.append('None')
    return(studentInfo)

def take_info_diplomas(data, v):
    studentInfo = []
    if data != []:
        for el in v:
            if el == 'voto' and data['lode'] == '1':
                studentInfo.append(101)
            # Scala il voto dei diplomi esteri
            elif el == 'voto' and data['baseVoto'] > '0' and data['baseVoto'] != '100':
                voto = round(
                    (float(data[el]) * 100) / int(data['baseVoto']))
                studentInfo.append(voto)
            elif el == 'voto' and data['diplomando'] != '1':
                # Gli indiani pur avendo base 100 hanno voti finali decimali
                studentInfo.append(round(float(data[el])))
            else:
                studentInfo.append(data[el])
    else:
        for el in v:
            studentInfo.append('None')
        print(studentInfo)
    return(studentInfo)

def take_info_isee(data, v):
    studentInfo = []
    if data != []:
        for el in v:
            studentInfo.append(data[el])
    else:
        for el in v:
            studentInfo.append('None')
    return studentInfo

def take_info_taken(data, v):
    studentInfo = []
    if data['esami'] != [] and data['esami'] != None:
        totCFU = 0
        lastDate = []
        grades = []
        for el in data['esami']:
            totCFU += el['cfu']
            lastDate.append(datetime.strptime(
                el['data'], '%d/%m/%Y')) if el['data'] != None else None
            grades.append(el['esito']['valoreNonNominale']
                        ) if not el['esito']['nominale'] else None  # Se è un'idoneità considera i crediti ma non va a cercare il voto
        studentInfo.append(totCFU)
        lastDate.sort()
        studentInfo.append(diff_month(lastDate[-1])) if lastDate != [
        ] else None  # min prende la piu' recente
        studentInfo.append(int(np.average(grades))) if grades != [
        ] else studentInfo.append(None)
    else:
        return [0, 12, 0]
    return studentInfo
    
def take_info_ll(data):
    if data != '':
        return diff_month(datetime.strptime(data, '%Y-%m-%dT%H:%M:%SZ'), 0)
    else:
        return 365 # 1 year -> back to May, 2021 when we didn't have lastLogin


### Creazione del dataset a partire dai json

In [5]:
######################## INSERISCE INTESTAZIONE NEL FILE ##########################################

vTot = ['ID_Stud']
for i in [vProfile, vIsee, vBooked, vTaken, vDiplomas]:
    for el in i:
        vTot.append(el)
vTot.append("lastLogin")
# vTot.append("Dropout")
with open('assets/out.tsv', 'w') as out_file:
    writer = csv.writer(out_file, delimiter='\t')
    writer.writerow(vTot)

######################## CREA FILE CSV ############################
# Se cambia il percorso di input bisogna stare attenti sopratutto allo spit nell'except
k = 0
listRaggr = []
for student in range(0, N_STUDENTS):
    studentInfo = [student]
    chkflag = True
    for t in types:
        file_name = p + str(student) + '.' + t
        try:  # Gestisce eccezione file not found
            with open(file_name) as in_data:
                tmp = json.load(in_data)
                if file_name == (p + str(student) + '.exams.booked.json'):
                    studentInfo.append(take_info_booked(tmp.pop('ritorno')))
                elif file_name == (p + str(student) + '.isee.json'):
                   studentInfo.append(take_info_isee(tmp, vIsee))
                elif file_name == (p + str(student) + '.profile.json'):
                    tmpchk = tmp.pop('ritorno')
                    if tmpchk != [] and tmpchk['accessoPrenotazioneNote'] != "Il soggetto non ha carriere aperte.":
                        studentInfo.append(take_info_profile(tmpchk, vProfile))
                    else:
                        chkflag = False
                        break
                elif file_name == (p + str(student) + '.diplomas.json'):
                    if tmp != None:
                        studentInfo.append(take_info_diplomas(tmp[0], vDiplomas))
                    else: studentInfo.append(['None' for _ in vDiplomas])
                elif file_name == (p + str(student) + '.exams.taken.json'):
                    tmp.pop('esito')
                    studentInfo.append(take_info_taken(tmp['ritorno'], vTaken))
        except Exception as e:
            f = str(e).split('.')[3:5]
            if f == ['exams', 'booked']: studentInfo.append([0, 6])
            elif f == ['isee', "json'"]: studentInfo.append(['None' for _ in vIsee])
            elif f == ['profile', "json'"]: studentInfo.append(['None' for _ in vProfile])
            elif f == ['diplomas', "json'"]: studentInfo.append(['None' for _ in vDiplomas])
            elif f == ['exams', 'taken']: studentInfo.append([0, 12, 0])
    try:
        with open((p + str(student) + '.lastlogin.txt')) as in_data:
            studentInfo.append(take_info_ll(in_data.read()))
    except:  # Exception as e:
        studentInfo.append(365)

    if chkflag:
        with open('assets/out.tsv', 'a') as out_file:
            writer = csv.writer(out_file, delimiter='\t')
            tmp = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, studentInfo) for i in b]
            writer.writerow(tmp)
    

#### Check

In [6]:
sentinels = ['None', 'n.d.', ' ', '']
df = pd.read_csv('assets/out.tsv', sep='\t', na_values=sentinels)
df.head()

,ID_Stud,annoAccaCors,annoCorso,annoDiNascita,cittadinanza,codCorso,creditiTotali,erasmus,facolta,flagIdentificato,...,valoreIntero,cfuPrenotati,ultPren,cfuTake,ultSup,mediaVoto,voto,diplomando,codiTiso,lastLogin
0,1,2022.0,1.0,21,ITA,29390.0,300.0,NO,ARCHITETTURA,1,...,-1.0,8.0,10,56.0,10.0,27.0,96.0,0.0,2346091.0,274.0
1,2,2022.0,6.0,28,Extra-UE,26637.0,180.0,NO,ECONOMIA,1,...,0.0,24.0,0,114.0,8.0,21.0,86.0,0.0,1783798.0,24.0
2,4,2022.0,1.0,27,ITA,30843.0,120.0,NO,INGEGNERIA CIVILE E INDUSTRIALE,0,...,3084.0,9.0,1,0.0,12.0,0.0,70.0,0.0,1880955.0,28.0
3,6,2022.0,1.0,23,ITA,31292.0,120.0,NO,"SCIENZE POLITICHE, SOCIOLOGIA, COMUNICAZIONE",1,...,28389.0,18.0,0,33.0,4.0,27.0,70.0,0.0,2183023.0,28.0
4,7,2021.0,2.0,28,ITA,15918.0,180.0,NO,INGEGNERIA CIVILE E INDUSTRIALE,1,...,NaN,9.0,17,177.0,17.0,22.0,NaN,NaN,NaN,365.0


In [7]:
sentinels = ['None', 'n.d.', ' ', '']
df = pd.read_csv('assets/out.tsv', sep='\t', na_values=sentinels)
df.shape

(85170, 30)

### Pulizia dati togliendo gli studenti di medicina, quelli in erasmus (incoming) e i diplomandi. Con relative colonne

In [8]:
with open('assets/cl_out.tsv', "w") as file:
    pass

with open('assets/out.tsv', mode='r') as file:
    csvFile = csv.reader(file, delimiter = '\t')
    for row in csvFile:
        if (row[7] == 'NO' or row[7] == 'erasmus') and (row[10] == 'False' or row[10] == 'isMedicina') and (row[28] == 'None' or row[27] == '0' or row[27] == 'diplomando') and (row[5] == 'codCorso' or row[5] != ''):
            row[9] = row[9].replace(",", " ")
            row[9] = row[9].replace("\n", " ")
            row[13] = row[13].replace(",", "-")
            row.pop(28)
            row.pop(27) #diplomando
            row.pop(10) #medicina
            row.pop(7) #erasmus
            with open('assets/cl_out.tsv', 'a') as out_file:
                writer = csv.writer(out_file, delimiter = '\t')
                writer.writerow(row)

In [2]:
sentinels = ['None', 'n.d.', ' ', '']
df = pd.read_csv('assets/cl_out.tsv', sep='\t', na_values=sentinels)
df.shape

(77034, 26)

### Etichettatura (dropout = true)

In [3]:
def supp_lab(row):
    res = True
    taken = float(row[21]) if row[21] != 'None' else 0
    booked = float(row[19]) if row[19] != 'None' else 0
    tot = float(row[6]) if row[6] != 'n.d' else 0
    if tot != 0 and (booked + taken) > tot - 30: # Ha finito (o quasi) il percorso
        res = False
    elif int(row[len(row)-1]) <= DELTA: # Se ha effettuato il login negli ultimi 6 mesi
            res = False  # NON dropout
    else:
        res = True

    return res

In [22]:
with open('assets/lab_out_2m.tsv', "w") as file:
    pass

with open('assets/cl_out.tsv', "r") as file:
    csvFile = csv.reader(file, delimiter = '\t')
    for row in csvFile:
        if row[len(row)-1] == 'lastLogin':
            row.append('Dropout')
        else:
            row.append(supp_lab(row))

        with open("assets/lab_out_2m.tsv", "a") as out:
            writer = csv.writer(out, delimiter = '\t')
            writer.writerow(row)

In [23]:
sentinels = ['None', 'n.d.', ' ', '']
df = pd.read_csv('assets/lab_out_2m.tsv', sep='\t', na_values=sentinels)
df.head()

,ID_Stud,annoAccaCors,annoCorso,annoDiNascita,cittadinanza,codCorso,creditiTotali,facolta,flagIdentificato,luogoDiNascita,...,nonDichiaro,valoreIntero,cfuPrenotati,ultPren,cfuTake,ultSup,mediaVoto,voto,lastLogin,Dropout
0,1,2022,1.0,21,ITA,29390,300,ARCHITETTURA,1,MARINO (RM),...,True,-1.0,8.0,10,56.0,10,27.0,96.0,274,True
1,2,2022,6.0,28,Extra-UE,26637,180,ECONOMIA,1,DURAZZO(ALBANIA),...,False,0.0,24.0,0,114.0,8,21.0,86.0,24,False
2,4,2022,1.0,27,ITA,30843,120,INGEGNERIA CIVILE E INDUSTRIALE,0,ROMA (RM),...,False,3084.0,9.0,1,0.0,12,0.0,70.0,28,False
3,6,2022,1.0,23,ITA,31292,120,"SCIENZE POLITICHE, SOCIOLOGIA, COMUNICAZIONE",1,ROMA (RM),...,False,28389.0,18.0,0,33.0,4,27.0,70.0,28,False
4,7,2021,2.0,28,ITA,15918,180,INGEGNERIA CIVILE E INDUSTRIALE,1,ROMA (RM),...,NaN,NaN,9.0,17,177.0,17,22.0,NaN,365,False


In [24]:
print(df.shape)
df = df[(pd.isnull(df['annoCorso']) == False) & (df['primaIscr'] <= (2022 - 2000)) & (df['annoDiNascita'] <= (2022 - 1970))]
df.to_csv('assets/lab_out_2m.tsv', index = False, sep='\t')
print(df.shape)

(77034, 27)
(76490, 27)


In [15]:
# v_dates = ["annoAccaCors", "annoDiNascita", "primaIscr", "ultIscr"]
# df = df.astype({'annoCorso': np.int32, "primaIscr": np.int32, "ultIscr": np.int32})
# for el in v_dates:
#     df[el] = df[el].apply(lambda x: int(round(time.mktime(datetime.strptime(str(x), '%Y').timetuple()))))

In [25]:
val = {"M": True, "F": False}
df["sesso"] = df["sesso"].map(val)

val = {"true": True, "false": False}
df["nonDichiaro"] = df["nonDichiaro"].map(val)

df = df.astype({'annoCorso': np.int32, 'annoAccaCors': np.int32, 'annoDiNascita': np.int32, 'sesso': bool, 'primaIscr': np.int32, 'ultIscr': np.int32,
                'cfuTake': np.int32, 'cfuPrenotati': np.int32, 'lastLogin': np.int64})

In [17]:
df.head()

,ID_Stud,annoAccaCors,annoCorso,annoDiNascita,cittadinanza,codCorso,creditiTotali,facolta,flagIdentificato,luogoDiNascita,...,nonDichiaro,valoreIntero,cfuPrenotati,ultPren,cfuTake,ultSup,mediaVoto,voto,lastLogin,Dropout
0,1,2022,1,21,ITA,29390,300,ARCHITETTURA,1,MARINO (RM),...,NaN,-1.0,8,10,56,10,27.0,96.0,274,True
1,2,2022,6,28,Extra-UE,26637,180,ECONOMIA,1,DURAZZO(ALBANIA),...,NaN,0.0,24,0,114,8,21.0,86.0,24,False
2,4,2022,1,27,ITA,30843,120,INGEGNERIA CIVILE E INDUSTRIALE,0,ROMA (RM),...,NaN,3084.0,9,1,0,12,0.0,70.0,28,False
3,6,2022,1,23,ITA,31292,120,"SCIENZE POLITICHE, SOCIOLOGIA, COMUNICAZIONE",1,ROMA (RM),...,NaN,28389.0,18,0,33,4,27.0,70.0,28,False
4,7,2021,2,28,ITA,15918,180,INGEGNERIA CIVILE E INDUSTRIALE,1,ROMA (RM),...,NaN,NaN,9,17,177,17,22.0,NaN,365,False


In [18]:
# print("Originale: ", df.shape[0])
# t = df[df['codCorso'] == 29923].shape[0] # Triennale
# print("Triennale", t)
# m = df[df['codCorso'] == 29932].shape[0] # Magistrale
# print("Magistrale: ", m)
# print("Totale: ", t+m)

In [19]:
# for el in [29923, 29932]:
#     index_names = df[df['codCorso'] == el].index
#     df.drop(index_names, inplace = True)
# df.rename(columns = {'voto':'votoDiploma'}, inplace = True)
# print(df.shape) # Ok 73226

In [26]:
print("Originale: ", df.shape[0])
i3s = df[df.facolta == "INGEGNERIA DELL'INFORMAZIONE, INFORMATICA E STATISTICA"]
print("I3S: ", i3s.shape[0])
print("Totale: ", df.shape[0] - i3s.shape[0])

Originale:  76490
I3S:  6471
Totale:  70019


In [27]:
index_names = df[df.facolta == "INGEGNERIA DELL'INFORMAZIONE, INFORMATICA E STATISTICA"].index
df.drop(index_names, inplace = True)
df.rename(columns = {'voto':'votoDiploma', 'annoDiNascita':'eta'}, inplace = True)
print(df.shape) # Ok 69967

(70019, 27)


In [28]:
df.loc[df.valoreIntero < 0,'valoreIntero'] = 50000
df = df.drop(['nonDichiaro'], axis=1)
index_names = df[df['mediaVoto'] > 31].index
df.drop(index_names, inplace=True)

In [29]:
print(df.shape)

(70018, 26)


In [30]:
df.to_csv('assets/yid_out_2m.tsv', index = False, sep='\t')

In [12]:
print("Dropout secondo criteri: ", sum(df.Dropout == True))
print("Non dropout: ", len(df)-sum(df.Dropout == True))

Dropout secondo criteri:  14222
Non dropout:  55796


## Per analisi finale

In [1]:
import json
import csv
from datetime import datetime
import numpy as np
import pandas as pd
from os import listdir

vTaken = ["cfuTake", "ultSup", "mediaVoto"]  # Esito -->"valoreNonNominale"


p = '../ex.taken/'
N_STUDENTS = max([int(el.split('.')[0]) for el in listdir(p)]) + 1

In [4]:
def take_info_taken(data, v):
    studentInfo = []
    if data['esami'] != [] and data['esami'] != None:
        totCFU = 0
        lastDate = []
        grades = []
        for el in data['esami']:
            totCFU += el['cfu']
            lastDate.append(datetime.strptime(
                el['data'], '%d/%m/%Y')) if el['data'] != None else None
            grades.append(el['esito']['valoreNonNominale']
                        ) if not el['esito']['nominale'] else None  # Se è un'idoneità considera i crediti ma non va a cercare il voto
        lastDate.sort()
        studentInfo.append(diff_month(lastDate[-1])) if lastDate != [
        ] else None  # min prende la piu' recente
        studentInfo.append(int(np.average(grades))) if grades != [
        ] else studentInfo.append(None)
    else:
        return [0, 12, 0]
    return studentInfo

In [29]:
studentInfo = {}
for file in listdir(p):
    student = file.split('.')[0]
    with open(p+file) as in_data:
        tmp = json.load(in_data)
        tmp.pop('esito')
        studentInfo[student] = take_info_taken(tmp['ritorno'], vTaken)

sentinels = ['None', 'n.d.', ' ', '']
df = pd.read_csv('assets/Infostud SDP - Studenti contattati.csv', sep=',', na_values=sentinels)
for k, v in studentInfo.items():
    # tmp = df[df["ID"] == el[0]]
    df.loc[df["ID"] == int(k), "UltimoSostenuto"] = v[0]
    df.loc[df["ID"] == int(k), "Media"] = v[1]

df['UltimoAccesso'] = df['UltimoAccesso'].apply(lambda x: diff_month(datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ')))
df.to_csv('assets/Infostud SDP - Studenti contattati.csv', index = False, sep=',')